In [1]:
import torch

from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.utils import to_dense_adj

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # print(adjacency.shape, x.shape)
        # print(x)
        x = torch.sparse.mm(adjacency, x)
        return x

In [4]:
dataset = CVFConfigDataset()
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [5]:
edge_index = torch.tensor([
    [0, 1],
    [0, 2],
    [0, 3],
    [1, 0],
    [2, 0],
    [3, 0]
], dtype=torch.long)

# # x = torch.tensor([[0], [0], [0], [0]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]
# x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

# y = torch.tensor([[3.0, 1.0]]) # ranks

# data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)
# data.train_mask = torch.tensor([1 for _ in range(len(x))])

In [6]:
adjacency = to_dense_adj(edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency

tensor([[1., 1., 1., 1.],
        [1., 1., 0., 0.],
        [1., 0., 1., 0.],
        [1., 0., 0., 1.]])

In [7]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)
    
    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.tanh(h)
        h = self.gnn2(h, adjacency)
        h = torch.tanh(h)
        h = self.out(h)
        return torch.sum(h, dim=0)
    
    def fit(self, data_loader, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs+1):
            for batch in data_loader:
                x = batch[0][0]
                y = batch[1]
                optimizer.zero_grad()
                out = self(x, adjacency)
                loss = criterion(out, y)
                loss.backward()
                optimizer.step()
            
            # if loss < 0.009:
            #     print("Loss threshold met.")
            #     break
            print("Loss:", loss)


In [8]:
gnn = VanillaGNN(1, 32, 1)
print(gnn)
gnn.fit(data_loader, epochs=50)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=1, out_features=32, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=32, out_features=32, bias=False)
  )
  (out): Linear(in_features=32, out_features=1, bias=True)
)
Loss: tensor(0.0189, grad_fn=<MseLossBackward0>)
Loss: tensor(0.1502, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0372, grad_fn=<MseLossBackward0>)
Loss: tensor(0.1043, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0659, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0636, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0326, grad_fn=<MseLossBackward0>)
Loss: tensor(0.1249, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0574, grad_fn=<MseLossBackward0>)
Loss: tensor(0.1177, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0644, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0800, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0112, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0312, grad_fn=<MseLossBackward0>)
Loss: tensor(0.0461, grad_fn=<MseLossBackward0>)

In [16]:
out = gnn(torch.tensor([[3.], [1.], [1.], [0.]]), adjacency)
print(out)

tensor([0.0318], grad_fn=<SumBackward1>)
